In [138]:
import yaml
import pandas as pd

In [144]:
match_ids = list()
with open(r"C:\Users\LENOVO\Downloads\ipl\README.txt", "r") as f:
    for line in (f.readlines()):
        if '1216492' in line:
            break
            
        if 'Royal Challengers' in line:
            match_ids.append(line.split("-")[6])

In [145]:
paths = list()
for mat_id in match_ids:
    gen_path = "C:/Users/LENOVO/Downloads/ipl/"
    ft =  ".yaml"
    str_id = (str(mat_id)).strip()
    path =  gen_path+str_id+ft
    paths.append(path)

In [146]:
documents=list()
for path in paths:
    with open(path) as file:
        document = yaml.full_load(file)
        documents.append(document)

In [184]:
bowlers = list()
kind = list()
non_striker = list()
overs = list()
total_runs = list()
extra_runs = list()
batsman_runs = list()
batsman = list()
player_dismissed = list()
venue = list()
for document in documents:
    for i in range(0, len(document['innings'])):
            for over in (list(document['innings'][i].values())[0]['deliveries']):
                for o, details in over.items():
                    overs.append(o)
                    non_striker.append(details['non_striker'])
                    total_runs.append(details['runs']['total'])
                    extra_runs.append(details['runs']['extras'])
                    batsman_runs.append(details['runs']['batsman'])   
                    batsman.append(details['batsman'])
                    bowlers.append(details['bowler'])
                    venue.append(document['info']['venue'])
                    if 'wicket' in list(details.keys()):
                        player_dismissed.append(1)
                        kind.append(details['wicket']['kind'])
                    else:
                        player_dismissed.append(0)
                        kind.append(0)

In [207]:
mi_team = ['RG Sharma', 'Q de Kock', 'SA Yadav', 'Ishan Kishan', 'HH Pandaya', 'KH Pandaya', 'NM Coulter-Nile', 'RD Chahar', 'JJ Bumrah', 'TA Boult']


[{0.1: {'non_striker': 'AJ Finch',
   'bowler': 'TA Boult',
   'runs': {'extras': 0, 'total': 2, 'batsman': 2},
   'batsman': 'D Padikkal'}},
 {0.2: {'non_striker': 'AJ Finch',
   'bowler': 'TA Boult',
   'runs': {'extras': 0, 'total': 4, 'batsman': 4},
   'batsman': 'D Padikkal'}},
 {0.3: {'non_striker': 'AJ Finch',
   'bowler': 'TA Boult',
   'runs': {'extras': 0, 'total': 0, 'batsman': 0},
   'batsman': 'D Padikkal'}},
 {0.4: {'non_striker': 'AJ Finch',
   'bowler': 'TA Boult',
   'runs': {'extras': 0, 'total': 1, 'batsman': 1},
   'batsman': 'D Padikkal'}},
 {0.5: {'non_striker': 'D Padikkal',
   'bowler': 'TA Boult',
   'runs': {'extras': 0, 'total': 0, 'batsman': 0},
   'batsman': 'AJ Finch'}},
 {0.6: {'non_striker': 'D Padikkal',
   'bowler': 'TA Boult',
   'runs': {'extras': 0, 'total': 1, 'batsman': 1},
   'batsman': 'AJ Finch'}},
 {1.1: {'non_striker': 'D Padikkal',
   'bowler': 'JL Pattinson',
   'runs': {'extras': 0, 'total': 2, 'batsman': 2},
   'batsman': 'AJ Finch'}},
 {

In [186]:
data = {"overs": overs, "batsman": batsman, "non-striker": non_striker,"bowler": bowlers, 'batsman_run': batsman_runs, 'extra_runs': extra_runs, 'total_runs': total_runs, 'player_dismissed': player_dismissed, 'kind':kind, 'venue':venue}

In [187]:
ipl = pd.DataFrame(data=data)
ipl.head()

,overs,batsman,non-striker,bowler,batsman_run,extra_runs,total_runs,player_dismissed,kind,venue
0,0.1,D Padikkal,AJ Finch,DL Chahar,0,0,0,0,0,Dubai International Cricket Stadium
1,0.2,D Padikkal,AJ Finch,DL Chahar,1,0,1,0,0,Dubai International Cricket Stadium
2,0.3,AJ Finch,D Padikkal,DL Chahar,0,2,2,0,0,Dubai International Cricket Stadium
3,0.4,AJ Finch,D Padikkal,DL Chahar,0,0,0,0,0,Dubai International Cricket Stadium
4,0.5,AJ Finch,D Padikkal,DL Chahar,4,0,4,0,0,Dubai International Cricket Stadium


In [202]:
out_bowler = ipl[(ipl['batsman']=='V Kohli') & (ipl['player_dismissed']==1)].bowler
bowler_type = ['Fast', 'Fast', 'Fast', 'Fast', 'Spin', 'Fast', 'Fast']
pd.DataFrame(data={'bowler': out_bowler, 'type':bowler_type})

,bowler,type
120,SM Curran,Fast
661,Kartik Tyagi,Fast
815,Mohammed Shami,Fast
1670,K Rabada,Fast
2025,RD Chahar,Spin
2349,SS Cottrell,Fast
2540,T Natarajan,Fast


In [206]:
ipl[(ipl['batsman']=='V Kohli') & (ipl['player_dismissed']==1)][['bowler', 'batsman', 'overs', 'kind']]

,bowler,batsman,overs,kind
120,SM Curran,V Kohli,18.8,caught
661,Kartik Tyagi,V Kohli,13.1,caught
815,Mohammed Shami,V Kohli,17.5,caught
1670,K Rabada,V Kohli,13.4,caught
2025,RD Chahar,V Kohli,12.2,caught
2349,SS Cottrell,V Kohli,2.4,caught
2540,T Natarajan,V Kohli,15.5,caught


In [208]:
ipl[(ipl['batsman']=='V Kohli') & (ipl['bowler'].isin(mi_team))]

,overs,batsman,non-striker,bowler,batsman_run,extra_runs,total_runs,player_dismissed,kind,venue
2013,10.2,V Kohli,D Padikkal,RD Chahar,1,0,1,0,0,Dubai International Cricket Stadium
2015,10.4,V Kohli,D Padikkal,RD Chahar,0,0,0,0,0,Dubai International Cricket Stadium
2016,10.5,V Kohli,D Padikkal,RD Chahar,0,0,0,0,0,Dubai International Cricket Stadium
2017,10.6,V Kohli,D Padikkal,RD Chahar,0,0,0,0,0,Dubai International Cricket Stadium
2019,11.2,V Kohli,D Padikkal,JJ Bumrah,0,0,0,0,0,Dubai International Cricket Stadium
2020,11.3,V Kohli,D Padikkal,JJ Bumrah,1,0,1,0,0,Dubai International Cricket Stadium
2022,11.5,V Kohli,D Padikkal,JJ Bumrah,0,0,0,0,0,Dubai International Cricket Stadium
2023,11.6,V Kohli,D Padikkal,JJ Bumrah,0,0,0,0,0,Dubai International Cricket Stadium
2025,12.2,V Kohli,D Padikkal,RD Chahar,0,0,0,1,caught,Dubai International Cricket Stadium
2204,0.2,V Kohli,AB de Villiers,JJ Bumrah,1,0,1,0,0,Dubai International Cricket Stadium
